In [114]:
import jax.numpy as np
import numpy as onp
from functools import partial
from jax import vmap
from jax.lax import scan
from jax.lax import cond
from jax import random
from jax import jit
from jax import jacrev
from jax.lax import stop_gradient
import matplotlib.pylab as plt
import matplotlib as mpl
from sklearn import manifold, datasets
import seaborn as sns
import jax

from jax.config import config
config.update("jax_debug_nans", True)

import jax.numpy as np
from utils import MidpointNormalize, load_data
from jax import random, flatten_util, vjp, jvp, custom_vjp, jacfwd, jacrev, vmap, grad
from IFD_tsne import tsne_fwd
import jax
import matplotlib.pylab as plt
import seaborn as sns

In [115]:
X, y = load_data(127)
key = random.PRNGKey(41)
#X = onp.array(random.normal(key, shape=(50, 50)))
y_guess = random.normal(key, shape=(X.shape[0], 2))
#Y_star = TSNE(n_components=2, learning_rate=200, init=onp.array(y_guess), perplexity=30).fit_transform(X)
Y_star = tsne_fwd(X, y_guess)

X_flat, X_unflattener = flatten_util.ravel_pytree(X)   # row-wise
Y_flat, Y_unflattener = flatten_util.ravel_pytree(Y_star) 

===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 0.03 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.01 seconds
===> Running optimization with exaggeration=12.00, lr=200.00 for 250 iterations...
Iteration   50, KL divergence 1.7257, 50 iterations in 0.2876 sec
Iteration  100, KL divergence 1.7844, 50 iterations in 0.3822 sec
Iteration  150, KL divergence 1.7430, 50 iterations in 0.3033 sec
Iteration  200, KL divergence 1.7834, 50 iterations in 0.2805 sec
Iteration  250, KL divergence 1.7445, 50 iterations in 0.2678 sec
   --> Time elapsed: 1.52 seconds
===> Running optimization with exaggeration=1.00, lr=200.00 for 750 iterations...
Iteration   50, KL divergence 0.5615, 50 iterations in 0.2973 sec
Iteration  100, KL divergence 0.5444, 50 iterations in 0.2792 sec
Iteration  150, KL divergence 0.5187, 50 iterations in 0.2729 sec
Iteration  200, KL divergence 0.5154, 50 iterations in 0.3176 sec
Iteration 

In [116]:
def logSoftmax(x):
    """Compute softmax for vector x."""
    max_x = np.max(x)
    exp_x = np.exp(x - max_x)
    sum_exp_x = np.sum(exp_x)
    log_sum_exp_x = np.log(sum_exp_x)
    max_plus_log_sum_exp_x = max_x + log_sum_exp_x
    log_probs = x - max_plus_log_sum_exp_x

    # Recover probs
    exp_log_probs = np.exp(log_probs)
    sum_log_probs = np.sum(exp_log_probs)
    probs = exp_log_probs / sum_log_probs
    return probs

def Hbeta(D: np.ndarray, beta=1.0):
    """
    Compute the log2(perplexity)=Entropy and the P-row (P_i) for a specific value of the
        precision=1/(sigma**2) (beta) of a Gaussian distribution. D: vector of squared Euclidean distances (without i)
    :param D: vector of length d, squared Euclidean distances to all other datapoints (except itself)
    :param beta: precision = beta = 1/sigma**2
    :return: H: log2(Entropy), P: computed probabilites
    """
    # TODO: exchange by softmax as described by https://nlml.github.io/in-raw-numpy/in-raw-numpy-t-sne/
    P = np.exp(-D * beta)     # numerator of p j|i
    sumP = np.sum(P, axis=None)    # denominator of p j|i --> normalization factor
    new_P = logSoftmax(-D * beta)
    sumP += 1e-8
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    return H, new_P

def Hbeta_final(D: np.ndarray, beta=1.0):
    """
    Compute the log2(perplexity)=Entropy and the P-row (P_i) for a specific value of the
        precision=1/(sigma**2) (beta) of a Gaussian distribution. D: vector of squared Euclidean distances (without i)
    :param D: vector of length d, squared Euclidean distances to all other datapoints (except itself)
    :param beta: precision = beta = 1/sigma**2
    :return: H: log2(Entropy), P: computed probabilites
    """
    # TODO: exchange by softmax as described by https://nlml.github.io/in-raw-numpy/in-raw-numpy-t-sne/
    P = np.exp(-D * beta)     # numerator of p j|i
    sumP = np.sum(P, axis=None)    # denominator of p j|i --> normalization factor
    sumP += 1e-8
    new_P = P/sumP
    return new_P

def HdiffGreaterTrue(*betas):
    beta, betamax = betas
    return beta*2

def HdiffGreaterFalse(*betas):
    beta, betamax = betas
    return (beta+betamax)/2

def HdiffSmallerTrue(*betas):
    beta, betamin = betas
    return beta/2

def HdiffSmallerFalse(*betas):
    beta, betamin = betas
    return (beta+betamin)/2

def HdiffGreater(*betas):
    beta, betamin, betamax = betas
    betamin = beta
    beta = cond((np.logical_or(betamax == np.inf, betamax == -np.inf)), HdiffGreaterTrue, HdiffGreaterFalse, *(beta, betamax))
    return beta, betamin, betamax

def HdiffSmaller(*betas):
    beta, betamin, betamax = betas
    betamax = beta
    beta = cond(np.logical_or(betamin == np.inf, betamin == -np.inf), HdiffSmallerTrue, HdiffSmallerFalse, *(beta, betamin))
    return beta, betamin, betamax

def HdiffGreaterTolerance(*betas):
    beta, betamin, betamax, Hdiff = betas
    beta, betamin, betamax = cond(Hdiff > 0, HdiffGreater, HdiffSmaller, *(beta, betamin, betamax))
    return beta, betamin, betamax, Hdiff

def binarySearch(res, el, Di, logU):
    print('Entered binary search function')
    Hdiff, thisP, beta, betamin, betamax = res
    beta, betamin, betamax, Hdiff = cond(np.abs(Hdiff) < 1e-5, lambda a, b, c, d: (a, b, c, d), HdiffGreaterTolerance, *(beta, betamin, betamax, Hdiff))
    (H, thisP) = Hbeta(Di, beta)
    Hdiff = H - logU
    return (Hdiff, thisP, beta, betamin, betamax), el

def x2beta_inner(Di: np.ndarray, iterator, beta, betamin, betamax, perplexity=30, tol=1e-5):
    """
    binary search for precision for Pi such that it matches the perplexity defined by the user
    :param Di: vector of length d-1, squared Euclidean distances to all other datapoints (except itself)
    :param beta: precision = beta = 1/sigma**2
    :return: final probabilites p j|i
    """
    # Compute the Gaussian kernel and entropy for the current precision
    logU = np.log(perplexity)
    H, thisP = Hbeta(Di, beta)
    Hdiff = H - logU

    print('Starting binary search')
    binarySearch_func = partial(binarySearch, Di=Di, logU=logU)

    # Note: the following binary Search for suitable precisions (betas) will be repeated 50 times and does not include the threshold value
    (Hdiff, thisP, beta, betamin, betamax), el = scan(binarySearch_func, init=(Hdiff, thisP, beta, betamin, betamax), xs=None, length=1000)    # Set the final row of P
    #thisP = np.insert(thisP, iterator, 0)
    return beta

def x2beta(D: np.ndarray, tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values (high-dim space) in such a way that each
        conditional Gaussian has the same perplexity.
    """
    # Initialize some variables
    n = D.shape[0]
    beta = np.ones(n)      # precisions (1/sigma**2)
    betamin = np.full(n, -np.inf)
    betamax = np.full(n, np.inf)
    betas_final = vmap(partial(x2beta_inner, perplexity=perplexity, tol=tol))(D, np.arange(n), beta=beta, betamin=betamin, betamax=betamax)
    return betas_final

def x2distance(X):
    print("Computing pairwise distances...")
    sum_X = np.sum(np.square(X), 1)
    (n, d) = X.shape
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    D = np.reshape(np.delete(D, np.array([i for i in range(0, D.shape[0]**2, (D.shape[0]+1))])), (n , n - 1 ))
    return D

def distance2p(D, betas):
    P_final = vmap(Hbeta_final, in_axes=0)(D, betas)
    #print('P_final', P_final, P_final.shape)
    P_final = vmap(partial(np.insert, values=0))(P_final, np.arange(P_final.shape[0]))
    return P_final

def y2q(Y: np.ndarray):
    # Compute pairwise affinities
    sum_Y = np.sum(np.square(Y), 1)
    num = -2. * np.dot(Y, Y.T)  # numerator
    num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
    num = num.at[np.diag_indices_from(num)].set(0.)     # numerator
    Q = num / np.sum(num)
    Q = np.maximum(Q, 1e-12)
    return Q, num

def KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener):
    """
    (R^nxp x R^nxp)--> R
    """
    X = X_unflattener(X_flat)
    Y = Y_unflattener(Y_flat)
    learning_rate, perplexity = (200, 30.0)
    D = x2distance(X)
    #print('D', D.shape)
    # first compute betas without tracking the derivative
    betas = x2beta(jax.lax.stop_gradient(D), tol=1e-5, perplexity=perplexity)
    #print('betas', betas.shape, betas)
    # use final betas to compute the probability matrix from the distances.
    # here D and therefor X is tracked for derivative computation
    P = distance2p(D, betas)
    P = (P + np.transpose(P))
    P = P / np.sum(P)      # Why don't we devide by 2N as described everywhere?
    P = np.maximum(P, 1e-12)
    #print('P', P, P.shape)
    Q, _ = y2q(Y)
    print('forward pass done')
    #print('Q', Q)
    return np.sum(P * (np.log(P+1e-10) - np.log(Q+1e-10)))

In [74]:
# fastes version for mixed Jacobian!!!
J_X_Y = jacrev(jacfwd(KL_divergence, argnums=1), argnums=0)(X_flat, Y_flat, X_unflattener, Y_unflattener)
print('J', J_X_Y)

Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done
J [[-1.14601536e-03  3.59309022e-04 -4.85297292e-04 ... -1.06886216e-06
   1.20807272e-06 -1.98809926e-06]
 [ 1.53055706e-03  6.50403148e-04 -8.21788795e-04 ... -1.92617995e-06
   1.47604638e-07 -7.97213318e-08]
 [-1.73236299e-06 -3.54500116e-05  2.48225697e-05 ...  2.09047312e-06
  -3.22121036e-06  3.82455983e-06]
 ...
 [-2.83922600e-06 -7.61937690e-06  1.07138885e-05 ... -3.11852745e-07
  -1.63817401e-06 -8.46495254e-07]
 [-5.08854373e-06  9.43038867e-06  1.20129052e-05 ... -7.50623731e-05
  -3.65586675e-06  3.23505665e-05]
 [-1.11801000e-05 -8.62797515e-06 -6.69115252e-08 ... -2.39926900e-04
  -2.02035408e-05 -2.45297619e-04]]


In [75]:
f = partial(KL_divergence, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
H = jax.hessian(f, argnums=1)(X_flat, Y_flat)
H_pinv = np.linalg.pinv(H + 1e-3*np.eye(len(H)), hermitian=True)

Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done


In [76]:
print(np.dot(-H_pinv, J_X_Y))

[[ 2.1048702e-01 -1.6090722e-01  2.2676721e-01 ...  9.1162761e-04
  -1.1729009e-03  2.4877361e-04]
 [-2.7798456e-01 -1.7961499e-01  2.2067337e-01 ...  4.7490635e-04
   2.9332246e-04  3.4087608e-04]
 [-3.6216539e-03  1.3330627e-02 -7.5492542e-03 ... -1.3888604e-03
   1.2332629e-03 -1.1181743e-03]
 ...
 [-5.4823514e-02 -4.8064124e-03  8.7097744e-03 ...  5.5088225e-04
  -2.1448886e-04 -7.4433578e-05]
 [-2.5935238e-03 -4.7630360e-03 -1.3998495e-02 ...  9.5764017e-03
   1.6811697e-02 -3.3119894e-03]
 [ 7.5441292e-03  1.4223445e-03  5.7380833e-04 ...  1.2731769e-02
   2.2200990e-02  3.9572187e-02]]


In [ ]:
def jvp_f(f, X_flat, Y_flat):
    return jax.jvp(f, (X_flat,), (Y_flat,))[1]

def vjp_f(X_flat):
    return jax.vjp(f, X_flat)[1]

In [4]:
KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener)

Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done


Array(0.51993895, dtype=float32)

In [117]:
def KL_divergence_dy(X_flat, Y_flat, X_unflattener, Y_unflattener):
    """
    (R^nxp x R^nxp)--> R
    """
    X = X_unflattener(X_flat)
    Y = Y_unflattener(Y_flat)
    learning_rate, perplexity = (200, 30.0)
    D = x2distance(X)
    #print('D', D.shape)
    # first compute betas without tracking the derivative
    betas = x2beta(jax.lax.stop_gradient(D), tol=1e-5, perplexity=perplexity)
    #print('betas', betas.shape, betas)
    # use final betas to compute the probability matrix from the distances.
    # here D and therefor X is tracked for derivative computation
    P = distance2p(D, betas)
    P = (P + np.transpose(P))
    P = P / np.sum(P)      # Why don't we devide by 2N as described everywhere?
    P = np.maximum(P, 1e-12)
    #print('P', P, P.shape)
    Q, num = y2q(Y)
    print('forward pass done')
    #print('Q', Q)

    PQ = P - Q
    PQ_exp = np.expand_dims(PQ, 2)  # NxNx1
    Y_diffs = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)  # nx1x2 - 1xnx2= # NxNx2
    num_exp = np.expand_dims(num, 2)    # NxNx1
    Y_diffs_wt = Y_diffs * num_exp
    return np.ravel(4 * np.sum((PQ_exp * Y_diffs_wt), axis=1))

In [118]:
f = partial(KL_divergence_dy, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
J_X_Y = jacrev(f, argnums=0)(X_flat, Y_flat)


Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done


In [80]:
f = partial(KL_divergence_dy, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
H = jax.jacrev(f, argnums=1)(X_flat, Y_flat)
H_pinv = np.linalg.pinv(H + 1e-3*np.eye(len(H)), hermitian=True)

Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done


In [81]:
print(H)

[[ 2.9759782e-03 -9.9696731e-04 -1.8315598e-06 ...  3.5689940e-04
  -1.7652492e-06  6.4417975e-08]
 [-9.9696754e-04  3.9306288e-03 -1.8981542e-05 ... -6.3432829e-04
   7.5418939e-07 -2.0860905e-06]
 [-1.8315550e-06 -1.8981529e-05  2.6589532e-03 ...  7.8229687e-07
   7.7389086e-07 -3.2129719e-06]
 ...
 [ 3.5689949e-04 -6.3432805e-04  7.8229840e-07 ...  4.9939742e-03
  -5.7279030e-06  8.8372599e-07]
 [-1.7652492e-06  7.5418211e-07  7.7388677e-07 ... -5.7279121e-06
   4.7008181e-03  2.2514626e-04]
 [ 6.4417520e-08 -2.0860907e-06 -3.2129719e-06 ...  8.8372633e-07
   2.2514630e-04  6.5454040e-03]]


In [97]:
f = partial(KL_divergence_dy, Y_flat=Y_flat, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
# jvp
_, jvp_fun_lin = jax.linearize(f, X_flat)
#jvp_fun = lambda v: jvp(f, (X_flat,), (v,))

# vjp
_, vjp_fun = vjp(f, X_flat)

Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done
Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done


In [98]:
import time
#time_start = time.time()
#print(vmap(jvp_fun_lin)(np.eye(len(X_flat))))
#time_end = time.time()
#print('time', time_end - time_start)

#time_start = time.time()
#print(vmap(jvp_fun)(np.eye(len(X_flat))))
#time_end = time.time()
#print('time', time_end - time_start)

# best version!!!!
time_start = time.time()
print(vmap(vjp_fun)(-H_pinv))
time_end = time.time()
print('time: ', time_end - time_start)

(Array([[ 2.1048674e-01, -1.6090760e-01,  2.2676829e-01, ...,
         9.1162801e-04, -1.1729077e-03,  2.4876936e-04],
       [-2.7798432e-01, -1.7961535e-01,  2.2067377e-01, ...,
         4.7489323e-04,  2.9334135e-04,  3.4088784e-04],
       [-3.6215934e-03,  1.3330657e-02, -7.5493040e-03, ...,
        -1.3888779e-03,  1.2332662e-03, -1.1181801e-03],
       ...,
       [-5.4823834e-02, -4.8064608e-03,  8.7097837e-03, ...,
         5.5088318e-04, -2.1448985e-04, -7.4437441e-05],
       [-2.5935331e-03, -4.7630374e-03, -1.3998534e-02, ...,
         9.5764417e-03,  1.6811693e-02, -3.3120278e-03],
       [ 7.5442195e-03,  1.4223302e-03,  5.7384721e-04, ...,
         1.2731675e-02,  2.2200927e-02,  3.9572086e-02]], dtype=float32),)
time:  0.37227821350097656


In [123]:
def compute_cov(vjp_fun, jvp_fun_lin, H_pinv_i, D, N, d, n):
  v1 = vjp_fun(-H_pinv_i)[0]
  print(v1)
  v2 = np.ravel(np.dot(np.dot(N, np.reshape(v1, (n, d), 'C')),np.transpose(D)), 'C')
  v3 = jvp_fun_lin(v2)
  return v3

In [128]:
from sklearn.datasets import make_spd_matrix
D = make_spd_matrix(X.shape[1])
N = make_spd_matrix(X.shape[0])

f = partial(KL_divergence_dy, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
H = jax.jacrev(f, argnums=1)(X_flat, Y_flat)
H_pinv = np.linalg.pinv(H + 1e-3*np.eye(len(H)), hermitian=True)

f = partial(KL_divergence_dy, Y_flat=Y_flat, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
# jvp
_, jvp_fun_lin = jax.linearize(f, X_flat)
# vjp
_, vjp_fun = vjp(f, X_flat)

time_start = time.time()
compute_cov_fun = lambda i: compute_cov(vjp_fun=vjp_fun, jvp_fun_lin=jvp_fun_lin, 
                                        H_pinv_i=i, D=D, N=N, d=D.shape[0], n=N.shape[0])
vmap(compute_cov_fun)(H_pinv)
time_end = time.time()
print('time: ', time_end - time_start)

Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done
Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done
Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done
Traced<ShapedArray(float32[6350])>with<BatchTrace(level=1/0)> with
  val = Array([[ 1.2624934e-01,  3.0275151e-01,  5.3603001e-02, ...,
         1.4042221e-04,  2.4873446e-04,  4.6364780e-04],
       [ 3.7767626e-02, -1.0878614e-01, -7.5483881e-02, ...,
        -1.5412066e-04, -2.1876383e-04, -2.6198086e-05],
       [ 8.1862500e-03, -8.4853005e-03,  3.2339965e-03, ...,
        -1.5797181e-05,  4.9772236e-04,  4.1241435e-04],
       ...,
       [ 7.1977056e-04,  5.2695966e-04, -1.2600278e-04, ...,
        -3.3954016e-03,  4.3024647e-04,  3.5436866e-03],
       [ 4.6736770e-04, -1.8354435e-04, -8.2751544e-04, ...,
        -4.0454887e-02,  8.8763125e-03,  1.4777500e-02],
       [ 1.

In [127]:
def compute_dy_dx(f, X, Y):
  H = jax.hessian(f, argnums=1)(X_flat, Y_flat)
  H_pinv = np.linalg.pinv(H + 1e-3*np.eye(len(H)), hermitian=True)
  J_X_Y = jacrev(jacfwd(f, argnums=1), argnums=0)(X_flat, Y_flat)
  return np.dot(-H_pinv, J_X_Y)

f = partial(KL_divergence, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
dy_dx = compute_dy_dx(f, X_flat, Y_flat)
print(dy_dx)
#np.dot(np.dot(dy_dx, np.kron(D, N)), dy_dx.T)

Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done
Computing pairwise distances...
Starting binary search
Entered binary search function
forward pass done
[[ 1.26248375e-01  3.02749753e-01  5.36025316e-02 ...  1.40419856e-04
   2.48732948e-04  4.63638135e-04]
 [ 3.77679951e-02 -1.08785085e-01 -7.54834563e-02 ... -1.54116351e-04
  -2.18762434e-04 -2.62033518e-05]
 [ 8.18620156e-03 -8.48526414e-03  3.23403650e-03 ... -1.57897048e-05
   4.97719739e-04  4.12389403e-04]
 ...
 [ 7.19800766e-04  5.26952499e-04 -1.26017112e-04 ... -3.39537626e-03
   4.30253975e-04  3.54363560e-03]
 [ 4.67383681e-04 -1.83537079e-04 -8.27537209e-04 ... -4.04547267e-02
   8.87632929e-03  1.47777274e-02]
 [ 1.85331574e-03  1.15959859e-03 -1.26393628e-03 ... -5.43645546e-02
  -1.84881256e-03  8.11846927e-02]]
